In [16]:
# Cell 1: Install Dependencies
# Run this cell first to install all the required Python packages.
# This might take a minute or two to complete.

print("--- Installing required libraries... ---")
!pip install -q streamlit pyngrok scikit-learn tensorflow opencv-python
print("✅ Libraries installed successfully!")

--- Installing required libraries... ---
✅ Libraries installed successfully!


In [35]:
%%writefile app.py
import streamlit as st
import os
import pandas as pd
import numpy as np
import pickle
from PIL import Image
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
import random
import cv2
import requests
from numpy.linalg import norm
from collections import Counter
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image

# --- 1. PAGE AND SESSION STATE CONFIGURATION ---
st.set_page_config(layout="wide", page_title="AI Fashion Finder")

# Initialize session state variables
if 'view' not in st.session_state: st.session_state.view = 'catalog'
if 'selected_item_idx' not in st.session_state: st.session_state.selected_item_idx = None
if 'history' not in st.session_state: st.session_state.history = []
if 'cart' not in st.session_state: st.session_state.cart = []
if 'weekly_planner' not in st.session_state:
    st.session_state.weekly_planner = {day: [] for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}
if 'style_profile' not in st.session_state:
    st.session_state.style_profile = {'likes': [], 'dislikes': [], 'recommendations': pd.DataFrame()}
if 'quiz_complete' not in st.session_state:
    st.session_state.quiz_complete = False


# --- 2. DATA AND MODEL LOADING (UNCHANGED) ---
@st.cache_resource
def download_haar_cascade():
    HAAR_CASCADE_PATH = "haarcascade_frontalface_default.xml"
    if not os.path.exists(HAAR_CASCADE_PATH):
        CASCADE_URL = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
        try:
            r = requests.get(CASCADE_URL); r.raise_for_status()
            with open(HAAR_CASCADE_PATH, 'wb') as f: f.write(r.content)
            return HAAR_CASCADE_PATH
        except: return None
    return HAAR_CASCADE_PATH

@st.cache_resource
def load_feature_extraction_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    model = Sequential([base_model, GlobalMaxPooling2D()])
    return model

BASE_DATA_DIR = "/kaggle/input/clothestry/clothes_tryon_dataset"
STYLE_CSV_PATH = "/kaggle/input/cloth-style-profile/cloth_style_profile.csv"
DESC_CSV_PATH = "/kaggle/input/cloth-description-profile/cloth_description_profile.csv"
FEATURES_PATH = "/kaggle/input/embeddings/embeddings_full_dataset.pkl"
MAP_PATH = "/kaggle/input/splitmap/file_split_map.pkl"
UPLOAD_DIR = "uploads"
if not os.path.exists(UPLOAD_DIR): os.makedirs(UPLOAD_DIR)
HAAR_CASCADE_PATH = download_haar_cascade()
DAYS_OF_WEEK = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

@st.cache_data
def load_data():
    for path in [STYLE_CSV_PATH, DESC_CSV_PATH, FEATURES_PATH, MAP_PATH]:
        if not os.path.exists(path): return pd.DataFrame()
    try:
        style_df = pd.read_csv(STYLE_CSV_PATH)
        desc_df = pd.read_csv(DESC_CSV_PATH)
        df = pd.merge(style_df, desc_df, on='filename', how='inner')
        with open(FEATURES_PATH, 'rb') as f: embeddings = list(pickle.load(f))
        file_map_df = pd.read_pickle(MAP_PATH)
        if len(embeddings) != len(file_map_df): return pd.DataFrame()
        file_map_df['resnet_embedding'] = embeddings
        final_df = pd.merge(df, file_map_df, on='filename', how='inner').reset_index(drop=True)
        final_df['cloth_path'] = final_df.apply(lambda r: os.path.join(BASE_DATA_DIR, r['split'], 'cloth', r['filename']), axis=1)
        final_df['model_path'] = final_df.apply(lambda r: os.path.join(BASE_DATA_DIR, r['split'], 'image', r['filename']), axis=1)
        final_df['search_text'] = (final_df['style_category'] + ' ' + final_df['description']).str.lower()
        return final_df
    except: return pd.DataFrame()

@st.cache_resource
def get_nn_model(_df):
    if _df.empty: return None
    return NearestNeighbors(n_neighbors=21, algorithm='brute', metric='cosine').fit(np.array(_df['resnet_embedding'].tolist()))

@st.cache_resource
def load_face_detector():
    if not HAAR_CASCADE_PATH: return None
    return cv2.CascadeClassifier(HAAR_CASCADE_PATH)


# --- 3. CORE FUNCTIONALITY (WITH NEW QUIZ RECOMMENDATION LOGIC) ---
def extract_features_from_image(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224)); img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    return result / norm(result)

def get_recommendations_by_vector(vector, nn_model, num_recs=20):
    distances, indices = nn_model.kneighbors([vector])
    return indices[0][:num_recs]

def display_interactive_results(results_df):
    if results_df.empty:
        st.info("No items match your specific criteria. Try broadening your search!")
        return
    cols = st.columns(5)
    for i, (idx, row) in enumerate(results_df.iterrows()):
        with cols[i % 5]:
            st.image(row['cloth_path'], use_container_width=True)
            st.caption(row['filename'])
            c1, c2 = st.columns(2)
            with c1:
                if st.button("🛒", key=f"rec_cart_{idx}", help="Add to Cart"):
                    if idx not in st.session_state.cart: st.session_state.cart.append(idx); st.success("Added!")
                    else: st.info("In cart")
            with c2:
                if st.button("👁️", key=f"rec_view_{idx}", help="View Details"):
                    st.session_state.selected_item_idx = idx; st.session_state.view = 'detail'; st.info("Go to 'Catalog' tab.")
                    st.rerun()

# ==========================================================
#  NEW: Smart Quiz Recommendation Engine
# ==========================================================
def get_recommendations_from_quiz(df, likes, dislikes, min_results=5):
    """
    Generates recommendations based on a list of liked and disliked keywords.
    """
    filtered_df = df.copy()

    # Negative filtering: remove all items that contain disliked keywords
    if dislikes:
        dislike_query = '|'.join(dislikes)
        filtered_df = filtered_df[~filtered_df['search_text'].str.contains(dislike_query, na=False)]

    # Positive filtering: find items that match liked keywords
    if likes:
        # Score items based on how many 'like' keywords they match
        filtered_df['score'] = filtered_df['search_text'].apply(lambda text: sum(1 for keyword in likes if keyword in text))
        
        # Prioritize items with more matches
        results = filtered_df[filtered_df['score'] > 0].sort_values(by='score', ascending=False)
    else:
        # If no likes, just use the dislike-filtered list
        results = filtered_df

    # Fallback: If results are too few, relax the criteria
    if len(results) < min_results:
        # If there were likes, try matching ANY liked term instead of scoring
        if likes:
            like_query = '|'.join(likes)
            results = filtered_df[filtered_df['search_text'].str.contains(like_query, na=False)]
    
    # Final Fallback: if still not enough, just show the most popular items from the dislike-filtered list
    if len(results) < min_results:
        results = filtered_df.head(min_results)

    return results.head(20) # Return up to 20 top results

# --- Other Core Functions (unchanged) ---
# ... (omitted for brevity)

# --- 4. LOAD DATA AND MODELS ---
df = load_data()
if df.empty: st.error("Application cannot start due to data loading issues."); st.stop()
resnet_neighbors = get_nn_model(df)
face_detector = load_face_detector()
feature_extractor = load_feature_extraction_model()

# --- 5. SIDEBAR NAVIGATION ---
st.sidebar.title("MENU"); cart_count = len(st.session_state.cart)
main_tab = st.sidebar.radio("Navigate", ["🏠 Home", "🛍️ Catalog & Search", "✨ My Style Profile", "🗓️ Weekly Planner", f"🛒 Cart ({cart_count})"])


# --- 6. UI: MAIN PAGE RENDERING ---
if main_tab == "🏠 Home":
    st.markdown('<h1>👗 AI Fashion Finder</h1>', unsafe_allow_html=True); st.markdown('<p>Welcome! Your smart fashion assistant is here. Use the menu on the left to explore our collection, get personalized style recommendations, plan your weekly outfits, and manage your shopping cart.</p>', unsafe_allow_html=True)
    st.image("https://images.unsplash.com/photo-1441986300917-64674bd600d8?q=80&w=2070&auto=format&fit=crop", use_container_width=True)

# ==========================================================
#  NEW: DEEP STYLE QUIZ IMPLEMENTATION
# ==========================================================
elif main_tab == "✨ My Style Profile":
    st.header("✨ Create Your AI-Powered Style Profile")
    
    quiz_tab, recs_tab, analysis_tab = st.tabs(["**Deep Style Quiz**", "**Your Recommendations**", "**Visual Search & Complexion**"])

    with quiz_tab:
        st.subheader("Help our AI understand your unique taste.")
        st.write("The more you tell us, the better your recommendations will be!")

        with st.form("deep_quiz_form"):
            st.markdown("#### Part 1: Your Go-To Styles")
            occasions = st.multiselect(
                "What occasions do you usually dress for? (Select all that apply)",
                ['Casual Day Out', 'Office & Work', 'Workout & Gym', 'Party & Evening', 'Formal Event', 'Vacation'],
                key='occasions'
            )

            st.markdown("#### Part 2: Colors & Patterns")
            colors = st.multiselect(
                "Which color palettes are you drawn to?",
                ['Neutral (black, white, beige)', 'Pastels (light pink, baby blue)', 'Earthy (brown, green, rust)', 'Jewel Tones (emerald, ruby, sapphire)', 'Bright & Bold (neon, primary colors)'],
                key='colors'
            )
            
            patterns = st.multiselect(
                "What about prints and patterns?",
                ['Solid Colors (No pattern)', 'Floral', 'Stripes', 'Polka Dots', 'Animal Print (leopard, snake)', 'Geometric'],
                key='patterns'
            )

            st.markdown("#### Part 3: What to Avoid")
            dislikes = st.multiselect(
                "Are there any styles or patterns you actively dislike?",
                ['Floral', 'Stripes', 'Polka Dots', 'Animal Print', 'Neon Colors', 'Oversized Fit', 'Tight Fit'],
                key='dislikes'
            )
            
            submitted = st.form_submit_button("Generate My Style Profile!", type="primary")

            if submitted:
                # --- Map quiz answers to keywords ---
                like_keywords = []
                # Map occasions and styles
                occasion_map = {'Casual Day Out': ['casual', 't-shirt', 'jeans'], 'Office & Work': ['blouse', 'trousers', 'formal'], 'Workout & Gym': ['activewear', 'sporty'], 'Party & Evening': ['dress', 'elegant', 'sequin']}
                for o in occasions: like_keywords.extend(occasion_map.get(o, []))
                
                # Map colors
                color_map = {'Neutral': ['black', 'white', 'beige', 'gray'], 'Pastels': ['pink', 'blue', 'lavender'], 'Earthy': ['green', 'brown', 'tan'], 'Jewel Tones': ['red', 'emerald', 'blue'], 'Bright & Bold': ['yellow', 'orange', 'neon']}
                for c in colors: like_keywords.extend(color_map.get(c, []))

                # Map patterns
                pattern_map = {'Solid Colors': ['solid', 'plain'], 'Floral': ['floral'], 'Stripes': ['stripe'], 'Polka Dots': ['dot'], 'Animal Print': ['leopard', 'zebra', 'snake'], 'Geometric': ['geometric', 'abstract']}
                for p in patterns: like_keywords.extend(pattern_map.get(p, []))
                
                # --- Map dislikes ---
                dislike_keywords = []
                dislike_map = {'Floral': ['floral'], 'Stripes': ['stripe'], 'Animal Print': ['leopard', 'zebra', 'snake'], 'Neon Colors': ['neon']}
                for d in dislikes: dislike_keywords.extend(dislike_map.get(d, []))
                
                st.session_state.style_profile['likes'] = list(set(like_keywords))
                st.session_state.style_profile['dislikes'] = list(set(dislike_keywords))
                
                recs = get_recommendations_from_quiz(df, st.session_state.style_profile['likes'], st.session_state.style_profile['dislikes'])
                st.session_state.style_profile['recommendations'] = recs
                st.session_state.quiz_complete = True
                st.success("Your profile has been created! Head over to the 'Your Recommendations' tab to see what we found for you.")
    
    with recs_tab:
        if not st.session_state.quiz_complete:
            st.info("⬅️ Please complete the Deep Style Quiz first to see your personalized recommendations.")
        else:
            st.subheader("✨ Here are some items we think you'll love")
            st.write(f"Based on your preferences for: **{', '.join(st.session_state.style_profile['likes'])}**")
            st.write(f"And avoiding: **{', '.join(st.session_state.style_profile['dislikes'])}**")
            st.markdown("---")
            
            recs_df = st.session_state.style_profile['recommendations']
            display_interactive_results(recs_df)

    with analysis_tab:
        # Visual Search and Complexion Analysis code remains the same
        st.subheader("Find it! Got a photo of clothing you love?")
        st.write("Upload an image of a garment, and our AI will find similar items in our catalog.")
        item_uploader = st.file_uploader("Upload clothing photo", type=['jpg','png','jpeg'], key="item_uploader")
        if item_uploader:
            fp = os.path.join(UPLOAD_DIR, item_uploader.name)
            with open(fp, "wb") as f: f.write(item_uploader.getbuffer())
            c1, c2 = st.columns([1, 2])
            with c1: st.image(fp, caption="Your Uploaded Image", use_container_width=True)
            with c2:
                with st.spinner("Analyzing style, color, and pattern..."):
                    uploaded_features = extract_features_from_image(fp, feature_extractor)
                    rec_indices = get_recommendations_by_vector(uploaded_features, resnet_neighbors, 20)
                    st.success(f"Found {len(rec_indices)} similar items!")
            st.markdown("---"); st.header("Here's what we found:")
            display_interactive_results(df.iloc[rec_indices])
            
        st.markdown("---")
        st.subheader("Get recommendations based on your unique coloring.")
        # ... (complexion analysis code is unchanged)

# --- Other Pages (Planner, Cart, Catalog) ---
# ... The code for these pages is the same as the last full version.
# I've pasted it here for completeness.
elif main_tab == "🗓️ Weekly Planner":
    st.header("🗓️ Your Weekly Outfit Planner"); st.info("Plan your outfits! Add items from the catalog's detail page.")
    if st.button("Clear Entire Plan", key="clear_plan"): st.session_state.weekly_planner = {d:[] for d in DAYS_OF_WEEK}; st.rerun()
    for day in DAYS_OF_WEEK:
        with st.expander(f"**{day}** ({len(st.session_state.weekly_planner[day])} items)"):
            if st.session_state.weekly_planner[day]:
                cols = st.columns(5)
                for i, idx in enumerate(st.session_state.weekly_planner[day]):
                    with cols[i % 5]:
                        st.image(df.loc[idx]['cloth_path'], use_container_width=True)
                        if st.button("❌", key=f"rem_plan_{day}_{idx}", help="Remove"): st.session_state.weekly_planner[day].remove(idx); st.rerun()
            else: st.caption("No outfit planned for this day.")

elif main_tab.startswith("🛒 Cart"):
    st.header(f"🛒 Shopping Cart ({cart_count} items)");
    if not st.session_state.cart: st.info("Your cart is empty. Add items from the catalog!")
    else:
        for i, idx in enumerate(st.session_state.cart):
            item = df.loc[idx]; col1, col2, col3 = st.columns([1, 4, 1])
            with col1: st.image(item['cloth_path'], width=100)
            with col2: st.subheader(f"{item['style_category']}"); st.caption(f"Item ID: {idx}")
            with col3:
                if st.button("Remove", key=f"rem_cart_{idx}"): st.session_state.cart.remove(idx); st.rerun()
            st.markdown("---")
        if st.button("Request All Items", type="primary", use_container_width=True): st.success("Demo: Your request would be sent to the store!"); st.balloons()

else: # Catalog & Search Tab
    if st.session_state.view == 'detail':
        idx = st.session_state.selected_item_idx; item = df.loc[idx]
        if st.button("⬅️ Back to Collection"): st.session_state.view = 'catalog'; st.rerun()
        c1, c2 = st.columns([1, 2])
        with c1: st.image(item['model_path'], use_container_width=True, caption="Model View")
        with c2:
            st.image(item['cloth_path'], use_container_width=True, caption="Garment View")
            st.header(item['style_category']); st.write(f"**AI Description:** *{item['description']}*")
            st.markdown("---")
            if st.button("Add to Cart", type="primary"):
                if idx not in st.session_state.cart: st.session_state.cart.append(idx); st.success("Added!"); st.rerun()
                else: st.info("Already in cart.")
            day_to_add = st.selectbox("Add to Planner:", ["- Select a Day -"] + DAYS_OF_WEEK, key=f"planner_add_{idx}")
            if day_to_add != "- Select a Day -":
                if idx not in st.session_state.weekly_planner[day_to_add]:
                    st.session_state.weekly_planner[day_to_add].append(idx); st.success(f"Added to {day_to_add}!")
                else: st.info(f"Already in {day_to_add}'s plan.")
        st.markdown("---"); st.header("✨ Visually Similar Items")
        if resnet_neighbors:
            distances, indices = resnet_neighbors.kneighbors(item['resnet_embedding'].reshape(1, -1))
            cols = st.columns(5)
            for i, rec_idx in enumerate(indices[0][1:6]):
                with cols[i]:
                    st.image(df.loc[rec_idx]['cloth_path'], caption=df.loc[rec_idx]['style_category'], use_container_width=True)
    else: # Catalog View
        st.markdown('<h1>🛍️ Catalog & Search</h1>', unsafe_allow_html=True)
        with st.expander("🔍 Search & Filter", expanded=True):
            col1, col2 = st.columns([2, 1])
            with col1: search_query = st.text_input("Search by description")
            with col2:
                style_options = ['All'] + sorted(df['style_category'].unique().tolist())
                selected_style = st.selectbox("Filter by Style Category:", style_options)
        filtered_df = df.copy()
        if selected_style != 'All': filtered_df = filtered_df[filtered_df['style_category'] == selected_style]
        if search_query: filtered_df = filtered_df[filtered_df['search_text'].str.contains(search_query.lower(), na=False)]
        st.header(f"Browse Our Collection ({len(filtered_df)} items found)")
        items_per_page = 15; total_pages = max(1, (len(filtered_df) - 1) // items_per_page + 1)
        page_num = st.number_input('Page', min_value=1, max_value=total_pages, value=1)
        start_idx = (page_num - 1) * items_per_page
        end_idx = page_num * items_per_page
        cols = st.columns(5)
        for i, (idx, row) in enumerate(filtered_df.iloc[start_idx:end_idx].iterrows()):
            with cols[i % 5]:
                st.image(row['cloth_path'], use_container_width=True)
                st.caption(f"{row['style_category']}")
                if st.button("View Details", key=f"view_{idx}"):
                    st.session_state.selected_item_idx = idx; st.session_state.view = 'detail'
                    if idx not in st.session_state.history: st.session_state.history.insert(0, idx)
                    st.session_state.history = st.session_state.history[:5]; st.rerun()

# --- SIDEBAR HISTORY ---
st.sidebar.markdown("---"); st.sidebar.header("🕰️ Recently Viewed")
if not st.session_state.history: st.sidebar.caption("No items viewed yet.")
else:
    for item_idx in st.session_state.history:
        item = df.loc[item_idx]
        if st.sidebar.button(f"{item['style_category']} (View)", key=f"history_{item_idx}", use_container_width=True):
            st.session_state.selected_item_idx = item_idx; st.session_state.view = 'detail'
            st.rerun()

Overwriting app.py


In [ ]:
import os
from pyngrok import ngrok
from kaggle_secrets import UserSecretsClient # Import the Kaggle secrets client

# --- IMPORTANT: Access the Ngrok token from Kaggle Secrets ---
# This is the correct way to use secrets in a Kaggle environment.
try:
    user_secrets = UserSecretsClient()
    NGROK_AUTH_TOKEN = user_secrets.get_secret("NGROK_AUTH_TOKEN")
    print("✅ Successfully loaded NGROK_AUTH_TOKEN from Kaggle Secrets.")
except:
    # This is a fallback in case the code is run outside Kaggle or the secret is missing
    NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTHTOKEN_IS_MISSING"
# -----------------


# Now, the rest of your code can proceed
if "MISSING" in NGROK_AUTH_TOKEN:
    print("\n🛑 ERROR: Could not find Kaggle Secret 'NGROK_AUTH_TOKEN'.")
    print("   Please ensure you have added your Ngrok token as a secret on the right-hand panel and attached it to this notebook.")
else:
    # Set the auth token for pyngrok
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # Check if app.py exists before trying to run it
    if not os.path.exists("app.py"):
        print("\n🛑 ERROR: 'app.py' not found. Please make sure you have successfully run the previous cell to create it.")
    else:
        try:
            # Start the streamlit app and connect it to a public URL
            public_url = ngrok.connect(8501)
            print("--- Launching the Streamlit application... ---")
            print("🎉 Your app is live!")
            print(f"🔗 Public URL: {public_url}")
            print("   (Note: The first time you open the app, it might take a moment to load.)")

            # This command runs the streamlit server. It will keep this cell running.
            !streamlit run app.py --server.port 8501 --server.headless true

        except Exception as e:
            print(f"\n🛑 An error occurred with ngrok or Streamlit: {e}")
            print("   Please double-check that your NGROK_AUTH_TOKEN secret is correct and that you have internet connectivity.")
            # Kill any existing ngrok tunnels if an error occurs
            ngrok.kill()

✅ Successfully loaded NGROK_AUTH_TOKEN from Kaggle Secrets.
--- Launching the Streamlit application... ---
🎉 Your app is live!
🔗 Public URL: NgrokTunnel: "https://4e851e194826.ngrok-free.app" -> "http://localhost:8501"
   (Note: The first time you open the app, it might take a moment to load.)



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://34.9.102.104:8501

2025-07-20 18:45:45.885649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753037145.915184     757 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753037145.924043     757 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when 